# Sequential Inference

This tutorial discusses particle filters, and sequential Monte Carlo techniques.

These techniques are relevant when performing inference on models where there is a large series of factor statements, some of which can be performed earlier than others. This situation often arises in time series models where the factor statements are the result of incoming observations, but the technique works for *any* probabilistic program.

As a motivating example, consider the following program:

In [2]:
ex :: MonadInfer m => m Bool
ex = replicateM 100 do
    ETC

: 

What distribution does this represent? It is the distribution over all lists of Booleans of length $10$, which places all the weight on the sequence `[True,True,True,True,True,True,True,True,True,True]`.

However, the naive approach to exactly inferring this distribution will not work. Why? Because it first constructs all $2^100$ possible solutions, and then throws away all but `[True,True,True,True,True,True,True,True,True,True]`. 

Now if we look at the structure of the program, it's clear that this is unnecessary. Each time a `condition` statement is made, we should throw away all possibilities with $0$ probability mass. If we do this, the size of the set of possible solutions never explodes.

We can perform this *sequential enumeration" with monad-bayes, as follows:

In [3]:
TODO

: 

`sis` is an inference method which performs a step of inference at each `factor` statement in the program. In the present case, we have used it in conjunction with exact inference, but the idea generalizes naturally to approximate inference methods.

To motivate this, let's examine a problem for which exact inference is no longer feasible, a non-linear state space model.



In [12]:
import Control.Monad.Bayes.Class
import Control.Monad.Bayes.Sampler
import qualified Graphics.Vega.VegaLite as VL
import IHaskell.Display.Hvega (vlShow)
import Control.Applicative

import Pipes (Producer, (>->))
import qualified Pipes as P
import Pipes.Prelude (unfoldr)
import qualified Pipes.Prelude as P
import Control.Monad.Bayes.Weighted

import Control.Monad.Bayes.Inference.SMC
import Control.Monad.Bayes.Inference.SMC2
import Control.Monad.Bayes.Inference.RMSMC
import Control.Monad.Bayes.Population
import Control.Monad.IO.Class

import Data.Ord
import Data.List
import Control.Monad
import Control.Arrow (first)
import Control.Monad.Bayes.Traced

:e OverloadedStrings

-- param :: MonadSample m => m (Double, Double)
-- param = do
--   let a = 0.01
--   let b = 0.01
--   precX <- gamma a b
--   let sigmaX = 1 / sqrt precX
--   precY <- gamma a b
--   let sigmaY = 1 / sqrt precY
--   return (sigmaX, sigmaY)

-- mean :: Double -> Int -> Double
-- mean x n = 0.5 * x + 25 * x / (1 + x * x) + 8 * cos (1.2 * fromIntegral n)

-- -- | A nonlinear series model from Doucet et al. (2000)
-- -- "On sequential Monte Carlo sampling methods" section VI.B
-- model ::
--   (MonadInfer m) =>
--   -- | observed data
--   [Double] ->
--   -- | prior on the parameters
--   (Double, Double) ->
--   -- | list of latent states from t=1
--   m [Double]
-- model obs (sigmaX, sigmaY) = do
--   let sq x = x * x
--       simulate [] _ acc = return acc
--       simulate (y : ys) x acc = do
--         let n = length acc
--         x' <- normal (mean x n) sigmaX
--         factor $ normalPdf (sq x' / 20) sigmaY y
--         simulate ys x' (x' : acc)
--   x0 <- normal 0 (sqrt 5)
--   xs <- simulate obs x0 []
--   return $ reverse xs

-- generateData ::
--   MonadSample m =>
--   -- | T
--   Int ->
--   -- | list of latent and observable states from t=1
--   m [(Double, Double)]
-- generateData t = do
--   (sigmaX, sigmaY) <- param
--   let sq x = x * x
--       simulate 0 _ acc = return acc
--       simulate k x acc = do
--         let n = length acc
--         x' <- normal (mean x n) sigmaX
--         y' <- normal (sq x' / 20) sigmaY
--         simulate (k -1) x' ((x', y') : acc)
--   x0 <- normal 0 (sqrt 5)
--   xys <- simulate t x0 []
--   return $ reverse xys

latentTransition :: MonadSample f => (Double, Double) -> f (Double, Double)
latentTransition (x,y) = liftA2 (,) ((+x) . (+0.5)  <$>  normal 0 1) ((+y) . (+0.5) <$> normal 0 1)


walk :: MonadSample m => Producer (Double,Double) m r
walk = P.unfoldr step2 (0,0)

resultOfWalk :: MonadSample m => m [(Double, Double)]
resultOfWalk =  P.fold  (\x y -> x <> [y]) [] id (walk >-> P.take 100)

toList :: MonadInfer m => P.Producer a m () -> m [a]
toList prod = P.fold  (\x y -> x <> [y]) [] id (prod >-> P.take 100)

observedWalk :: MonadInfer m => m [(Double, Double, Double, Double)]
observedWalk = toList (walk >-> P.mapM (\(x,y) -> do
    x' <- (normal x 1) 
    y' <- (normal y 1)
    return (x,y,x',y')))

step2 s = Right <$> do
    new <- (latentTransition s)
    return (new, new)

scatter xs ys xs' ys' =
  let encoding = VL.encoding
            . VL.position VL.X [VL.PName "X", VL.PmType VL.Quantitative] --, VL.PScale [VL.SDomain $ VL.DNumbers [xmin, xmax]]]
            . VL.position VL.Y [VL.PName "Y", VL.PmType VL.Quantitative] -- , VL.PScale [VL.SDomain $ VL.DNumbers [ymin, ymax]]]
            . VL.color [VL.MName "Year"]
      dat = (VL.dataFromColumns [ ] 
                . VL.dataColumn "X" (VL.Numbers $ xs <> xs')
                . VL.dataColumn "Y" (VL.Numbers $ ys <> ys')
                . VL.dataColumn "Year" (VL.Strings (replicate (length xs) "First" <> replicate (length xs) "Second"))
                ) []
  in VL.toVegaLite [ 
              dat,
              VL.mark VL.Circle [VL.MColor "green"]
              , encoding []
              , VL.width 400
              , VL.height 400
              ]



In [13]:
(xs, ys,xs',ys') <- unzip4 <$> (sampleIO $ prior observedWalk) -- (sampleIO $ generateData 5)

vlShow $ scatter xs ys xs' ys'

(1.3849012051408411,1.3188133567229836)

In [42]:


conditioning :: (MonadSample m, MonadCond m) => P.Producer ((Double,Double), (Double,Double)) m ()
conditioning = P.zip walk (P.each (zip xs' ys')) >-> P.chain (\((x,y), (x',y')) -> factor (normalPdf x 1 x' ))





-- smcrmRess <- sampleIO $ runPopulation $ rmsmcLocal 10 10 10 (toList conditioning)
-- smc2Res <- sampleIO $ runPopulation $ smc2 100 3 2 1 param (model ys)
smcRes <- sampleIO $ runPopulation $ smcMultinomial 100 5000 (toList conditioning)

In [15]:

trSamples <- sampleIO $ prior $ mh 20000 $ toList conditioning

In [37]:
-- :!ghc-pkg list
-- :load ../src/Ising

newtype Point = Point {toPair :: [(Double,Double)]}

instance Num Point where
    Point p1 + Point p2 = Point (zipWith (\(x,y)  (x',y') -> ((x+x')/2,(y+y')/2) ) p1 p2)
    fromInteger (_) = undefined

Line 8: Redundant bracket
Found:
(_)
Why not:
_

In [40]:


(infX, infY) = unzip $ (toPair . foldr1 (+)  . fmap Point) [( ( fmap (\((a,b),(c,d)) -> (a,b)))) $ fst $ x | x <- smcRes]

-- (infX, infY) = unzip $ ( ( fmap (\((a,b),(c,d)) -> (a,b)))) $ fst $ head $ sortOn (Down . snd) smcRes


vlShow $ scatter xs ys infX infY


Line 3: Use sum
Found:
foldr1 (+)
Why not:
sumLine 3: Redundant bracket
Found:
((fmap (\ ((a, b), (c, d)) -> (a, b)))) $ fst $ x
Why not:
fmap (\ ((a, b), (c, d)) -> (a, b)) $ fst $ xLine 3: Redundant $
Found:
fst $ x
Why not:
fst x

In [41]:
(infX, infY) = unzip $ (toPair . foldr1 (+) . fmap Point) [( ( fmap (\((a,b),(c,d)) -> (a,b)))) $ x | x <- take 10 trSamples]

-- (infX, infY) = unzip $ ( ( fmap (\((a,b),(c,d)) -> (a,b)))) $ fst $ head $ sortOn (Down . snd) smcRes


vlShow $ scatter xs ys infX infY

Line 1: Use sum
Found:
foldr1 (+)
Why not:
sumLine 1: Redundant bracket
Found:
((fmap (\ ((a, b), (c, d)) -> (a, b)))) $ x
Why not:
fmap (\ ((a, b), (c, d)) -> (a, b)) $ xLine 1: Use <$>
Found:
((fmap (\ ((a, b), (c, d)) -> (a, b)))) $ x
Why not:
(\ ((a, b), (c, d)) -> (a, b)) <$> xLine 1: Redundant $
Found:
((fmap (\ ((a, b), (c, d)) -> (a, b)))) $ x
Why not:
((fmap (\ ((a, b), (c, d)) -> (a, b)))) x

To perform approximate inference here, we can represent a distribution as a *population* of weighted samples, and at each factor statement, do a resampling operation to keep the population healthy. This is **Sequential Monte Carlo**. Here it is in code:

The algorithm creates a population of $n$ samples (initially weighted equally) and updates them at each factor statement. The update is `smcMultinomial`, which samples new samples from the empirical distribution defined by the population. More sophisticated resamplers are available, including `smcSystematic` (see TODO)

In [43]:
:load ../src/Ising

size


10.0